In [2]:
#Клонируем данные из репозитория

!git clone https://github.com/ai4se-course/ai4se-hse-course-24-25.git

Cloning into 'ai4se-hse-course-24-25'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 22 (delta 3), reused 1 (delta 1), pack-reused 7 (from 1)
Receiving objects: 100% (22/22), 9.56 KiB | 9.56 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [3]:
#Устанавливаем необходимые зависимости

!pip install -r /content/ai4se-hse-course-24-25/02-func-name-suggestion/requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.6/542.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Create

In [35]:
#Импортируем load_dataset из библиотеки datasets для загрузки данных

from datasets import load_dataset

dataset = load_dataset('code_search_net', 'python', split='test', trust_remote_code=True)

#Выберем первую тысячу записей и переназначим переменную dataset
dataset = dataset.select(range(1000))

In [36]:
#Выведем полученный датасет

print(dataset)

#Как видим, датасет содержит 1000 строк и необходимые нам features

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 1000
})


In [37]:
##Обработка данных

#После успешной загрузки данных, их необходимо обработать, выделить названия функций, тела с комментариями и документации и без них

from ast import Str
from tree_sitter import Language, Parser
from tree_sitter_python import language

#Утсановим язык python для парсера
PY_LANGUAGE = Language(language())

# init парсера
parser = Parser(PY_LANGUAGE)

In [38]:
#Перейдем к непосредственному написанию обработчика, документация к каждой функции находится непосредственно внутри



def extract_function_data(source_code: str):
    """
    Извлекает данные о функции из исходного кода, включая:
    - Имя функции.
    - Тело функции с комментариями.
    - Тело функции без комментариев и строк.

    Args:
        source_code (str): Исходный код.

    Returns:
        tuple: (имя функции, тело с комментариями, тело без комментариев).
    """
    tree = parser.parse(source_code.encode('utf8'))
    root = tree.root_node

    # Инициализация
    function_name = None
    function_body_with_comments = None
    ranges_to_remove = []

    # Поиск определения функции
    for node in root.children:
        if node.type == 'function_definition':
            function_name = extract_function_name(node)
            function_body_with_comments = extract_node_text(source_code, node.child_by_field_name('body'))
            ranges_to_remove += find_removal_ranges(node)

    # Удаление комментариев и строк
    cleaned_source_code = remove_ranges_from_code(source_code, ranges_to_remove)

    # Повторный анализ дерева для извлечения "чистого" тела функции
    cleaned_tree = parser.parse(cleaned_source_code.encode('utf8'))
    root = cleaned_tree.root_node
    function_body_without_comments = None

    for node in root.children:
        if node.type == 'function_definition':
            function_body_without_comments = extract_node_text(cleaned_source_code, node.child_by_field_name('body'))

    return function_name, function_body_with_comments, function_body_without_comments


def extract_function_name(function_node):
    """
    Извлекает имя функции из узла дерева.

    Args:
        function_node: Узел дерева типа 'function_definition'.

    Returns:
        str: Имя функции.
    """
    name_node = function_node.child_by_field_name('name')
    return name_node.text.decode('utf8') if name_node else None


def extract_node_text(source_code: str, node):
    """
    Извлекает текст узла из исходного кода.

    Args:
        source_code (str): Исходный код.
        node: Узел дерева.

    Returns:
        str: Текст узла.
    """
    if not node:
        return None
    return source_code[node.start_byte:node.end_byte]


def find_removal_ranges(node):
    """
    Находит диапазоны байтов для удаления комментариев и строк.

    Args:
        node: Узел дерева.

    Returns:
        list: Список кортежей (start_byte, end_byte) для удаления.
    """
    ranges = []

    if node.type == 'comment' or (node.type == 'string' and node.parent.type == 'expression_statement'):
        ranges.append((node.start_byte, node.end_byte))

    for child in node.children:
        ranges += find_removal_ranges(child)

    return ranges


def remove_ranges_from_code(source_code: str, ranges_to_remove):
    """
    Удаляет указанные диапазоны байтов из исходного кода.

    Args:
        source_code (str): Исходный код.
        ranges_to_remove (list): Список диапазонов (start_byte, end_byte).

    Returns:
        str: Очищенный исходный код.
    """
    source_code_list = list(source_code)
    for start, end in reversed(ranges_to_remove):
        del source_code_list[start:end]

    # Удаление пустых строк
    cleaned_code = ''.join(source_code_list)
    return '\n'.join(line for line in cleaned_code.splitlines() if line.strip())



In [39]:
#Проверим как отработает наш код на sample_code из задания:

sample_code = """
def sina_xml_to_url_list(xml_data):
    \"\"\"str->list
    Convert XML to URL List.
    From Biligrab.
    \"\"\"
    rawurl = []
    # Comment1
    # Comment 2
    dom = parseString(xml_data)
    for node in dom.getElementsByTagName('durl'):
        url = node.getElementsByTagName('url')[0]  # Comment 3
        rawurl.append(url.childNodes[0].data)
    return rawurl
"""

function_name, function_body_with_comments, function_body_without_comments = extract_function_data(sample_code)

print(f"Function Name: {function_name}")
print("Fuction body with comments:")
print(function_body_with_comments)
print("Function body without comments:")
print(function_body_without_comments)

#Отлично! Обработчик работает корректно, теперь можно обучать модель

Function Name: sina_xml_to_url_list
Fuction body with comments:
"""str->list
    Convert XML to URL List.
    From Biligrab.
    """
    rawurl = []
    # Comment1
    # Comment 2
    dom = parseString(xml_data)
    for node in dom.getElementsByTagName('durl'):
        url = node.getElementsByTagName('url')[0]  # Comment 3
        rawurl.append(url.childNodes[0].data)
    return rawurl
Function body without comments:
rawurl = []
    dom = parseString(xml_data)
    for node in dom.getElementsByTagName('durl'):
        url = node.getElementsByTagName('url')[0]  
        rawurl.append(url.childNodes[0].data)
    return rawurl


In [40]:
#Прежде чем начнем обучение модели, необходимо обработать искомый датасет

def add_elements_to_dataset(dataset):
    dataset = dataset.map(lambda x: {
        'function_name': extract_function_data(x['whole_func_string'])[0],
        'function_body_with_comments': extract_function_data(x['whole_func_string'])[1],
        'function_body_without_comments': extract_function_data(x['whole_func_string'])[2]
    })
    return dataset

In [41]:
dataset = add_elements_to_dataset(dataset)

print(dataset.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url', 'function_name', 'function_body_with_comments', 'function_body_without_comments']


In [42]:
#Отлично, датасет обработан успешно, столбцы добавлены. Теперь выберем несколько рандомных функций из датасета и проверим, что получилось:
import random

start = random.randint(1, 995)
end = start + 5

for i in range(start, end):
  print(f"whole_func_string:\n{dataset[i]['whole_func_string']}")
  print(f"function_name:\n{dataset[i]['function_name']}")
  print(f"function_body_with_comments:\n{dataset[i]['function_body_with_comments']}")
  print(f"function_body_without_comments:\n{dataset[i]['function_body_without_comments']}")
  print('__________________________________________________________________________________________')
  print('__________________________________________________________________________________________')

whole_func_string:
def render_log_filename(ti, try_number, filename_template):
    """
    Given task instance, try_number, filename_template, return the rendered log
    filename

    :param ti: task instance
    :param try_number: try_number of the task
    :param filename_template: filename template, which can be jinja template or
        python string template
    """
    filename_template, filename_jinja_template = parse_template_string(filename_template)
    if filename_jinja_template:
        jinja_context = ti.get_template_context()
        jinja_context['try_number'] = try_number
        return filename_jinja_template.render(**jinja_context)

    return filename_template.format(dag_id=ti.dag_id,
                                    task_id=ti.task_id,
                                    execution_date=ti.execution_date.isoformat(),
                                    try_number=try_number)
function_name:
render_log_filename
function_body_with_comments:
"""
    Given task instan

In [43]:
#Теперь запустим обучение через T5


from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "Salesforce/codet5p-220m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [44]:
print(dataset[1])

{'repository_name': 'soimort/you-get', 'func_path_in_repository': 'src/you_get/extractors/miomio.py', 'func_name': 'sina_xml_to_url_list', 'whole_func_string': 'def sina_xml_to_url_list(xml_data):\n    """str->list\n    Convert XML to URL List.\n    From Biligrab.\n    """\n    rawurl = []\n    dom = parseString(xml_data)\n    for node in dom.getElementsByTagName(\'durl\'):\n        url = node.getElementsByTagName(\'url\')[0]\n        rawurl.append(url.childNodes[0].data)\n    return rawurl', 'language': 'python', 'func_code_string': 'def sina_xml_to_url_list(xml_data):\n    """str->list\n    Convert XML to URL List.\n    From Biligrab.\n    """\n    rawurl = []\n    dom = parseString(xml_data)\n    for node in dom.getElementsByTagName(\'durl\'):\n        url = node.getElementsByTagName(\'url\')[0]\n        rawurl.append(url.childNodes[0].data)\n    return rawurl', 'func_code_tokens': ['def', 'sina_xml_to_url_list', '(', 'xml_data', ')', ':', 'rawurl', '=', '[', ']', 'dom', '=', 'parse

In [47]:
from collections.abc import Iterable
from functools import cache
from pprint import pprint

import evaluate
import datasets


@cache
def _init_metrics():
    return (evaluate.load("exact_match"), evaluate.load("rouge"))


def predict(dataset: datasets.Dataset, model, tokenizer) -> None:
    """
    Предсказывает имена функций и оценивает результаты.
    """
    predictions = [
        model_predict(f"def :\n    {example['function_body_without_comments']}", model, tokenizer)
        for example in dataset
    ]
    references = [example["function_name"] for example in dataset]

    # Нормализация
    predictions = [p.strip().lower() for p in predictions]
    references = [r.strip().lower() for r in references]

    # Оценка метрик
    eval_results = run_evaluate(predictions=predictions, references=references)

    # Вывод результатов
    print()
    print("*" * 80)
    print("Evaluation results:")
    pprint(eval_results)
    print("*" * 80)
    print()


def model_predict(input_text: str, model, tokenizer) -> str:
    """
    Выполняет предсказание имени функции с использованием модели.

    Args:
        input_text (str): Входной текст (тело функции).
        model: Предобученная модель.
        tokenizer: Токенизатор модели.

    Returns:
        str: Предсказанное имя функции.
    """
    # Исправлено: используем input_text
    input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
    with torch.no_grad():  # Отключение вычисления градиентов
        outputs = model.generate(input_ids, max_length=20)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Очистка предсказания
    if '(' in prediction:
        prediction = prediction.split('(')[0]
    prediction = prediction.strip()
    if ' ' in prediction:
        splitted = prediction.split(' ')
        prediction = splitted[1] if splitted[0] == 'def' else splitted[0]
    return prediction


def run_evaluate(
    predictions: Iterable[str], references: Iterable[str]
) -> dict[str, float]:
    """
    Оценивает метрики Exact Match и ROUGE.

    Args:
        predictions (Iterable[str]): Предсказанные имена функций.
        references (Iterable[str]): Эталонные имена функций.

    Returns:
        dict[str, float]: Результаты оценки.
    """
    em, rouge = _init_metrics()
    em_score = em.compute(predictions=predictions, references=references)
    rouge_scores = rouge.compute(predictions=predictions, references=references)

    return {**rouge_scores, **em_score}


# Предсказание и оценка
predict(dataset, model=model, tokenizer=tokenizer)



********************************************************************************
Evaluation results:
{'exact_match': 0.002,
 'rouge1': 0.04656150793650793,
 'rouge2': 0.01610357142857143,
 'rougeL': 0.04687460317460315,
 'rougeLsum': 0.04671071428571427}
********************************************************************************



In [49]:
# Теперь аналогичная процедура для тела с комментариями:

from collections.abc import Iterable
from functools import cache
from pprint import pprint

import evaluate
import datasets


@cache
def _init_metrics():
    return (evaluate.load("exact_match"), evaluate.load("rouge"))


def predict(dataset: datasets.Dataset, model, tokenizer) -> None:
    """
    Предсказывает имена функций и оценивает результаты.
    """
    predictions = [
        model_predict(f"def :\n    {example['function_body_with_comments']}", model, tokenizer)
        for example in dataset
    ]
    references = [example["function_name"] for example in dataset]

    # Нормализация
    predictions = [p.strip().lower() for p in predictions]
    references = [r.strip().lower() for r in references]

    # Оценка метрик
    eval_results = run_evaluate(predictions=predictions, references=references)

    # Вывод результатов
    print()
    print("*" * 80)
    print("Evaluation results:")
    pprint(eval_results)
    print("*" * 80)
    print()


def model_predict(input_text: str, model, tokenizer) -> str:
    """
    Выполняет предсказание имени функции с использованием модели.

    Args:
        input_text (str): Входной текст (тело функции).
        model: Предобученная модель.
        tokenizer: Токенизатор модели.

    Returns:
        str: Предсказанное имя функции.
    """
    # Исправлено: используем input_text
    input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
    with torch.no_grad():  # Отключение вычисления градиентов
        outputs = model.generate(input_ids, max_length=20)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Очистка предсказания
    if '(' in prediction:
        prediction = prediction.split('(')[0]
    prediction = prediction.strip()
    if ' ' in prediction:
        splitted = prediction.split(' ')
        prediction = splitted[1] if splitted[0] == 'def' else splitted[0]
    return prediction


def run_evaluate(
    predictions: Iterable[str], references: Iterable[str]
) -> dict[str, float]:
    """
    Оценивает метрики Exact Match и ROUGE.

    Args:
        predictions (Iterable[str]): Предсказанные имена функций.
        references (Iterable[str]): Эталонные имена функций.

    Returns:
        dict[str, float]: Результаты оценки.
    """
    em, rouge = _init_metrics()
    em_score = em.compute(predictions=predictions, references=references)
    rouge_scores = rouge.compute(predictions=predictions, references=references)

    return {**rouge_scores, **em_score}


# Предсказание и оценка
predict(dataset, model=model, tokenizer=tokenizer)



********************************************************************************
Evaluation results:
{'exact_match': 0.017,
 'rouge1': 0.07696179653679651,
 'rouge2': 0.027938095238095238,
 'rougeL': 0.07709534632034631,
 'rougeLsum': 0.07669058441558442}
********************************************************************************

